In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html

In [64]:
df = pd.read_csv('/Users/jacksonsorenson/Documents/Pyhton Projects/AB:Testing/ab_testing.csv')
df.head()

,User ID,Group,Page Views,Time Spent,Conversion,Device,Location
0,14292,B,3,424,No,Mobile,Northern Ireland
1,11682,A,9,342,No,Mobile,Scotland
2,19825,A,2,396,No,Desktop,Northern Ireland
3,16080,B,4,318,No,Desktop,Wales
4,18851,A,1,338,Yes,Desktop,Scotland


In [65]:
time_spent = df.groupby('Group')['Time Spent'].sum().reset_index()
conversions = df.groupby(['Group', 'Conversion']).size().unstack().fillna(0).reset_index()
device_distribution = df.groupby(['Group', 'Device']).size().unstack().fillna(0).reset_index()
location_distribution = df.groupby(['Group', 'Location']).size().unstack().fillna(0).reset_index()

In [69]:
fig_time_spent = px.bar(time_spent, x='Group', y='Time Spent',
                        title='Total Time Spent by Campaign Type',
                        labels={'Group': 'Campaign Type', 'Time Spent': 'Time Spent (seconds)'})

In [70]:
fig_conversions = px.bar(conversions, x='Group', y='Yes',
                         title='Conversions by Campaign Type',
                         labels={'Group': 'Campaign Type', 'Yes': 'Number of Conversions'})

In [71]:
fig_device_distribution = go.Figure()
for device in device_distribution.columns[1:]:
    fig_device_distribution.add_trace(go.Bar(
        x=device_distribution['Group'], y=device_distribution[device], name=device))
fig_device_distribution.update_layout(
    title='Device Usage Distribution by Campaign Type',
    xaxis_title='Campaign Type', yaxis_title='Count', barmode='stack')

In [72]:
fig_location_distribution = go.Figure()
for location in location_distribution.columns[1:]:
    fig_location_distribution.add_trace(go.Bar(
        x=location_distribution['Group'], y=location_distribution[location], name=location))
fig_location_distribution.update_layout(
    title='Location Distribution by Campaign Type',
    xaxis_title='Campaign Type', yaxis_title='Count', barmode='stack')

In [73]:
app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Campaign Analysis Dashboard", style={'textAlign': 'center'}),

    dcc.Graph(figure=fig_time_spent),
    dcc.Graph(figure=fig_conversions),
    dcc.Graph(figure=fig_device_distribution),
    dcc.Graph(figure=fig_location_distribution)
])

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True)